<h1> Материалы курса "Анализ данных в инфромационной безопасности" </h1>

<h2> Практическое занятие № 2.2 </h2>

<h2> Тема: "Python+SQLite" </h2>

<h2>Автор:</h2>

<p>Синадский Алексей</p>
<p>Исследователь исследовательского центра UDV Group</p>
<p>aleksei.sinadskiy@udv.group</p>


Содержание:
- Чтение данных
- Создание БД SQLite
- Расследование
- (допишите)
- то
- сё
- пятое
- десятое
- Выводы

# Легенда


## Письмо-задание

Добрый день, уважаемые коллеги!
Вчера поступило сообщение об ограблении банкоматов банка «Надежный» в городе Калиниград. Цели расположены по адресам: улица Александра Невского, 193А, улица Кирова, 28 (Лицей № 49) и улица Октябрьская, 68-74. По предварительной информации, полученной от сотрудников безопасности банка, работала группа налетчиков в количестве более трех человек.
Прошу оказать содействие в расследовании инцидента и выявить группу злоумышленников.

P.S. Напоминаю, что в отчете необходимо указать алгоритм поиска и полностью воспроизвести цепочку событий.
С уважением.


## Рекомендации от ведущего аналитика

Старшие коллеги (aka 5 курс) решают эту задачу без дополнительной информации. Тем не менее, для ускорения процесса рекомендую следующую последовательность действий:

    Знакомство
    a. Посмотреть, какие данные доступны. Скачать CSV, создать БД (можно использовать SQLite)
    b. Посмотреть, как работать с данными в SQLite.
    с. Для анализа использовать запросы к SQLite на SQL.
    Привязка геокоординат
    a. Посмотреть активные LAC и CellID, построить таблицу соответствий
    b. Почитать про MCC-MNC коды (https://ru.wikipedia.org/wiki/MNC)
    c. Получить координаты базовых станций по известным LAC и CellID, отметить их на Google Maps метками. Для поиска координат можно использовать внешний сервис (раньше работал https://cellidfinder.com/cells, сейчас можно попробовать https://www.opencellid.org или другой).
    d. По известным в письме адресам выбрать вероятные точки, входящие в маршрут злоумышленников, по точкам выбрать вероятные базовые станции.
    Поиск последовательно перемещающихся абонентов
    a. Для всех абонентов, которые были на каждой из интересующих точек, выбрать последнее время посещения (можно использовать вспомогательные таблицы)
    b. Из всех абонентов из запроса выше выбрать таких, которые последовательно побывали на всех трёх адресах, т.е. на следующий приходили в более позднее время.
    c. Считать их потенциальными злоумышленниками («подозреваемыми»)
    Поиск связей
    a. Найти «друзей подозреваемых», т.е. всех, кто звонил и кому звонили «подозреваемые» (список уникальных абонентов)
    b. Найти пересечения списков «друзей подозреваемых» и «подозреваемых»
    Поиск маршрутов (или водителя-наблюдателя)
    a. Поискать таких абонентов, которые были на тех же точках с большим допуском на время. Может быть, он связывался по результатам преступления с кем-то.
    С уважением, Ведущий Аналитик.

## Уточнение от организатора соревнования

В связи с недоступностью известных сетевых ресурсов по обнаружению географических координат базовых станций по LAC и CellID и некорректной работой доступных аналогичных сервисов у ряда студентов возникли сложности с выполнением задачи "найти LAC и CellID по адресу".

Технически это решаемо использованием API от Google с подменой личности, но это непрофильная для раздела про SQL задача, поэтому в лабораторной работе предлагаю использовать следующие LAC и CellID для каждого из мест:

    Улица Александра Невского, 193А: LAC 2212, CellID 47805
    Улица Кирова, 28 (Лицей № 49): [(LAC 3907, CellID 3134), (LAC 2212, CellID 37111), (LAC 2212, CellID 47536)]
    Улица Октябрьская, 68-74: LAC 3907, CellID 39781

Для адреса "2" БС несколько, т.к. области их действия перекрываются.

Несмотря на то, что я привожу искомые LAC и CellID, в лабораторной работе необходимо описать методику их поиска по геокоординатам и привести примеры для тех базовых станций, которые получится обнаружить, и только после этого использовать приведённую выше подсказку.

# Импорт нужных модулей

In [1]:
import pandas as pd  # для обработки данных в Python
import sqlite3  # лёгкая БД - подробнее посмотреть можно тут: https://habr.com/ru/articles/754400/
import contextlib  # для удобного открытия и закрытия подключеия к БД

# Путь к файлам

Нужно скачать файлы из соревнвания Kaggle

In [2]:
data_billing_types = '../2024/kaggle студенты/billing_type.csv'
data_persons = '../2024/kaggle студенты/persons.csv'
data_main_billing = '../2024/kaggle студенты/main_billing.csv'
db_file = 'billing.db'

# Запись данных в БД

Простой вариант - прочитать вcё сразу в ОЗУ. Но данные бывают разного размера, поэтому лучше записывать поэтапно (или просто сразу средствами SQLite).

Общий план - прочитать данные, создать таблицу, заполнить

In [3]:
chunksize = 1000  # для тренировки - как будто много данных и не входит в ОЗУ

## Типы биллинга

In [4]:
df = pd.read_csv(data_billing_types, nrows=10)
df.head(3)

,id,billing_type
0,1,SMS Out
1,2,GPRS
2,3,Normal LocUpd


In [5]:
with contextlib.closing(sqlite3.connect(db_file)) as connection:
    with contextlib.closing(connection.cursor()) as cursor:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS billing_types (
                       id INTEGER PRIMARY KEY,
                       billing_type TEXT NOT NULL
            )
            '''
        )

In [6]:
if True:
    with contextlib.closing(sqlite3.connect(db_file)) as connection:
        for chunk in pd.read_csv(data_billing_types, chunksize=chunksize):
            chunk.to_sql('billing_types', connection, if_exists='append', index=False)
        connection.commit()

## Абоненты

In [7]:
df = pd.read_csv(data_persons, nrows=10)
df.head(3)

,id,soc_st,surname,name,patronymic,move_template,mobile_operator,lac_id,cell_id,imsi,imei,msisdn
0,1,51,Захаров,Дмитрий,Ярославович,0,4,1000,10021,250204722322322,11612001753630,79922233222
1,2,52,Захарова,София,Игоревна,0,3,2212,37035,250283722322323,358758053490950,79032233223
2,3,53,Захаров,Егор,Дмитриевич,2,4,1000,10621,250203622322333,355248034227300,79022233233


Создание таблицы для пользователей

In [8]:
with contextlib.closing(sqlite3.connect(db_file)) as connection:
    with contextlib.closing(connection.cursor()) as cursor:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS persons (
                       Допишите код
            )
            '''
        )

Запись информации о пользователях в таблицу

In [9]:
if True:
    with contextlib.closing(sqlite3.connect(db_file)) as connection:
        # допишите тут
        connection.commit()

## Сам биллинг

In [10]:
df = pd.read_csv(data_main_billing, nrows=10)
df.head(3)

,id,billtime,call_duration,billing_type_id,lac_id,cell_id,phone_b,abonent_id,abonentimei,abonentimsi,abonentphone,reverseabonentphone,reversephoneb
0,1,2014-06-06 00:00:00,0,1,3907,3024,79222245677,2269,12542001965610,250281522334802,79052245705,0,0
1,2,2014-06-06 00:00:00,2,1,3907,39781,79922253992,3780,359206055062750,250023622343098,79222254003,0,0
2,3,2014-06-06 00:00:00,1,1,3901,39762,79022352703,4102,353160043883270,250280322344864,79052255766,0,0


Создание таблицы для биллинга

In [11]:
# Допишите тут

Заполнение таблицы с биллингом

In [12]:
# Допишите тут

## Результат

Посмотрим результат

In [13]:
with contextlib.closing(sqlite3.connect(db_file)) as connection:
    data = connection.execute('''SELECT * FROM billing_types''').fetchall()
data

[(1, 'SMS Out'),
 (2, 'GPRS'),
 (3, 'Normal LocUpd'),
 (4, 'Call Out'),
 (5, 'SMS In'),
 (6, 'Call In')]

In [14]:
with contextlib.closing(sqlite3.connect(db_file)) as connection:
    data = connection.execute('''SELECT * FROM main_billing LIMIT 1''').fetchall()
data

[(1,
  '2014-06-06 00:00:00',
  0,
  1,
  3907,
  3024,
  '79222245677',
  2269,
  '12542001965610',
  '250281522334802',
  '79052245705',
  '0',
  '0')]

# Расследование

Тут начинается самое интересное - нужно проанализировать таблицу и понять, кто есть преступник. Напомню, начать можно с LAC и CellID

In [15]:
with contextlib.closing(sqlite3.connect(db_file)) as connection:
    data = connection.execute('''SELECT abonentphone FROM main_billing WHERE lac_id == 2212 LIMIT 5''').fetchall()
data

[('79122425621',),
 ('79122428306',),
 ('79022428420',),
 ('79222431479',),
 ('79052233251',)]

## Подготовка итогового ответа

Сдавать данные на Kaggle нужно в определённом формате. Эта секция поможет вам подготовить данные для загрузки в систему оценивания.

Загрузим исходные данные

In [16]:
df_answer = pd.read_csv(data_persons, usecols=['id', 'surname',	'name', 'patronymic'])
df_answer.head(3)

,id,surname,name,patronymic
0,1,Захаров,Дмитрий,Ярославович
1,2,Захарова,София,Игоревна
2,3,Захаров,Егор,Дмитриевич


Добавим идентификатор для Kaggle и базвую разметку - "все невиновны" (у всех "0")

In [17]:
df_answer['ID'] = df_answer.apply(lambda row: str(row['id']) + '_' + row['surname'] + '_' + row['name'] + '_' + row['patronymic'], axis=1)
df_answer['label'] = 0
df_answer.head(3)

,id,surname,name,patronymic,ID,label
0,1,Захаров,Дмитрий,Ярославович,1_Захаров_Дмитрий_Ярославович,0
1,2,Захарова,София,Игоревна,2_Захарова_София_Игоревна,0
2,3,Захаров,Егор,Дмитриевич,3_Захаров_Егор_Дмитриевич,0


Тут надо разметить по id тех людей, которых вы считаете преступниками (указать для них label "1")

In [18]:
list_of_criminals_id = [0, 3, 5]  # сюда нужно добавить идентификаторы преступников
df_answer.loc[df_answer['id'].isin(list_of_criminals_id), 'label'] = 1
df_answer.head(6)

,id,surname,name,patronymic,ID,label
0,1,Захаров,Дмитрий,Ярославович,1_Захаров_Дмитрий_Ярославович,0
1,2,Захарова,София,Игоревна,2_Захарова_София_Игоревна,0
2,3,Захаров,Егор,Дмитриевич,3_Захаров_Егор_Дмитриевич,1
3,4,Захаров,Данил,Дмитриевич,4_Захаров_Данил_Дмитриевич,0
4,5,Захарова,Арина,Дмитриевна,5_Захарова_Арина_Дмитриевна,1
5,6,Морозов,Григорий,Юрьевич,6_Морозов_Григорий_Юрьевич,0


Удалим лишние колонки

In [19]:
df_answer = df_answer.drop(columns=['id', 'surname', 'name', 'patronymic'])
df_answer.head(3)

,ID,label
0,1_Захаров_Дмитрий_Ярославович,0
1,2_Захарова_София_Игоревна,0
2,3_Захаров_Егор_Дмитриевич,1


Подготовим файл для загрузки в Kaggle

In [20]:
df_answer.to_csv('my_answer.csv', index=False)

# Выводы

Здесь напишите выводы по выполненной работе.